In [8]:
import pickle as pkl
from copy import deepcopy
import jsonlines
from utils.my_util import cluster_mentions, remove_speaker_prefix
import json

In [9]:
speaker_dict = {}
with open('data/raw_source/dialogue_en/all_coref_data_en_finalized.json', 'r') as f:
    temp = json.load(f)
    for line in temp:
        scene_id = line['scene_id']
        speakers = []
        for sent in line['sentences']:
            speakers.append(" ".join(sent[:sent.index(":")]))
        speaker_dict[scene_id] = speakers

split_dict = {"train":[], "dev":[], "test":[]}
with open('data/raw_source/dialogue_zh/dev_temp.pkl', 'rb') as f:
    temp = pkl.load(f)
    for line in temp:
        split_dict['dev'].append(line['scene_id'])
with open('data/raw_source/dialogue_zh/test_temp.pkl', 'rb') as f:
    temp = pkl.load(f)
    for line in temp:
        split_dict['test'].append(line['scene_id'])
with open('data/raw_source/dialogue_zh/train_temp.pkl', 'rb') as f:
    temp = pkl.load(f)
    for line in temp:
        split_dict['train'].append(line['scene_id'][:-1])

In [10]:
data = []
all_ids = []
with open('data/raw_source/dialogue_en/all_coref_data_en_finalized.json', 'r') as f:
# with open('data/raw_source/dialogue_zh/dev-test-batch1_zh.json', 'r') as f:
    reader = jsonlines.Reader(f)
    for bulk in reader:
        for idx, instance in enumerate(bulk):
            if idx>=5:
                break

            scene_id = instance['scene_id']
            if scene_id == "":
                continue
            sentences = instance['sentences']

            for sent in sentences:
                print(sent.index(":"), sent)
            # print(sentences)
            # sentences = [[token for token in "".join(sent)] for sent in instance['sentences']]
            annotations = instance['annotations']
            all_ids.append(scene_id)
            speakers = speaker_dict[scene_id]
            answers = []
            for item in annotations:
                query = (item['query']['sentenceIndex'], item['query']['startToken']-1-sentences[item['query']['sentenceIndex']].index(":"), item['query']['endToken']-1-sentences[item['query']['sentenceIndex']].index(":"))
                antecedents = item['antecedents']
                if antecedents in [['n', 'o', 't', 'P', 'r', 'e', 's', 'e', 'n', 't'], ['null_projection'], ['empty_subtitle']]:
                    answers.append([query, "notPresent"])
                else:
                    temp_answer = []
                    for antecedent in antecedents:
                        if isinstance(antecedent, dict):
                            temp_answer.append((antecedent['sentenceIndex'], antecedent['startToken']-1-sentences[antecedent['sentenceIndex']].index(":"), antecedent['endToken']-1-sentences[antecedent['sentenceIndex']].index(":")))
                        else:
                            temp_answer = " ".join(antecedents)
                    answers.append([query, temp_answer])
            new_sentences = []
            for sent in sentences:
                new_sentences.append(sent[sent.index(":")+1:])

            data.append({
                "sentences": new_sentences,
                "answers": answers,
                "speakers": speakers,
                "scene_id": scene_id
            })

1 ['Sheldon', ':', 'If', 'a', 'photon', 'is', 'directed', 'through', 'a', 'plane', 'with', 'two', 'slits', 'in', 'it', 'and', 'either', 'is', 'observed', 'it', 'will', 'not', 'go', 'through', 'both', '.', 'If', 'unobserved', ',', 'it', 'will', '.', 'If', 'it', "'s", 'observed', 'after', 'it', 'left', 'the', 'plane', ',', 'before', 'it', 'hits', 'its', 'target', '...', 'it', 'will', 'not', 'have', 'gone', 'through', 'both', 'slits', '.']
1 ['Leonard', ':', 'Agreed', '.', 'What', "'s", 'your', 'point', '?']
1 ['Sheldon', ':', 'There', "'s", 'no', 'point', ',', 'I', 'just', 'think', 'it', "'s", 'a', 'good', 'idea', 'for', 'a', 'T', '-', 'shirt', '.']
1 ['Leonard', ':', 'Excuse', 'me', '.']
1 ['Receptionist', ':', 'Hang', 'on', '.']
1 ['Leonard', ':', 'One', 'across', 'is', 'Aegean', ',', 'eight', 'down', 'is', 'Nabokov', '.', 'Twenty', '-', 'six', 'across', 'is', 'MCM', '.', 'Fourteen', 'down', 'is', '...', 'Move', 'your', 'finger', '.', 'phylum', ',', 'which', 'makes', '14', 'across', 'P

In [16]:
split = "train"

for i in range(len(data)):
    if i >= 20:
        continue
    sample = data[i]
    scene_id = sample['scene_id']
    # if sample['scene_id'] not in split_dict[split]:
    #     continue
    print(scene_id)
    # print(sample)
    # print()
    # original_sentences = sample['sentences']
    # original_clusters = cluster_mentions(sample['answers'], original_sentences)
    # sentences, clusters, speakers = remove_speaker_prefix(original_sentences, original_clusters)
    sentences = sample['sentences']
    clusters = cluster_mentions(sample['answers'], sentences)
    speakers = sample['speakers']
    print(clusters)


s01e01c00t
[[(0, 1, 3)], [(0, 6, 8)], [(0, 52, 54), (0, 22, 23), (0, 9, 11)], [(0, 12, 13), (0, 7, 8), (0, 37, 39)], [(0, 35, 36), (0, 14, 15), (0, 2, 3), (0, 41, 42), (0, 43, 44), (0, 31, 32), (0, 46, 47), (0, 17, 18), (0, 27, 28)], [(0, 43, 45)], [(1, 2, 3)], [(1, 4, 6)], [(0, -2, -1), (1, 4, 5)], [(2, 0, 1)], [(1, 5, 6), (2, 2, 4)], [(2, -2, -1), (2, 5, 6)], [(2, 8, 9), (2, 10, 13)], [(2, 14, 18)], [(3, -2, -1), (3, 1, 2)], [(5, 3, 4)], [(5, 8, 9)], [(5, 15, 16)], [(5, 22, 24), (4, -2, -1), (5, 22, 23)], [(5, 25, 26)], [(5, 47, 52), (5, 31, 36), (5, 45, 46), (5, 39, 44)], [(5, 53, 54)], [(6, 1, 2), (6, -2, -1)], [(5, -2, -1), (6, 3, 4)], [(7, 6, 12), (7, 5, 6)], [(8, 7, 8), (7, -2, -1), (8, 1, 2)], [(9, -2, -1), (9, 0, 1)], [(9, 2, 3), (9, 4, 6), (7, 7, 12)], [(10, 1, 2)], [(10, -2, -1), (11, 1, 2)], [(12, 3, 4), (11, 3, 4)], [(12, 3, 5)], [(12, 10, 11), (12, -2, -1), (12, 6, 7)], [(12, 10, 13)], [(13, -2, -1), (14, 6, 7), (13, 2, 3), (13, 6, 7), (14, 3, 4)], [(13, 9, 10)], [(14, 8,